# Bibliotecas

In [459]:
import pandas as pd
import json
import numpy as np
from datetime import date
import random
from unidecode import unidecode

In [460]:
pd.set_option('display.max_columns', None)

# Dicionários

## Nível profissional

In [461]:
dict_nivel_profissional = {
    'Aprendiz': 1
    ,'Trainee': 2
    ,'Auxiliar': 3
    ,'Assistente': 4
    ,'Técnico de Nível Médio': 5
    ,'Júnior': 6
    ,'Analista': 7
    ,'Pleno': 8
    ,'Sênior': 9
    ,'Especialista': 10
    ,'Líder': 11
    ,'Supervisor': 12
    ,'Coordenador': 13
    ,'Gerente': 14
}

## Nível linguagem

In [462]:
dict_nivel_linguagem = {
    'Nenhum': 0
    ,'Básico': 1
    ,'Intermediário': 2
    ,'Técnico': 3
    ,'Avançado': 4
    ,'Fluente': 5
}

## Nível escolaridade

In [463]:
dict_nivel_escolaridade = {
    'Ensino Fundamental Incompleto': 1
    ,'Ensino Fundamental Completo': 2
    ,'Ensino Médio Incompleto': 3
    ,'Ensino Médio Completo': 4
    ,'Ensino Técnico Incompleto': 5
    ,'Ensino Técnico Cursando': 6
    ,'Ensino Técnico Completo': 7
    ,'Ensino Superior Incompleto': 8
    ,'Ensino Superior Cursando': 9
    ,'Ensino Superior Completo': 10
    ,'Pós Graduação Incompleto': 11
    ,'Pós Graduação Cursando': 12
    ,'Pós Graduação Completo': 13
    ,'Mestrado Incompleto': 14
    ,'Mestrado Cursando': 15
    ,'Mestrado Completo': 16
    ,'Doutorado Incompleto': 17
    ,'Doutorado Cursando': 18
    ,'Doutorado Completo': 19
}

# Prospects

## Lendo e carregando o arquivo json em um dataframe

In [464]:
with open('C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/Base de dados/prospects.json', 'r', encoding='utf-8') as f:
    dados_prospects = json.load(f)

lista_prospects = []

for vaga_id, vaga_info in dados_prospects.items():
    vaga_info['id_vaga'] = vaga_id
    lista_prospects.append(vaga_info)

df_prospects_raw = pd.DataFrame(lista_prospects)

df_prospects_raw.head(5)

,titulo,modalidade,prospects,id_vaga
0,CONSULTOR CONTROL M,,"[{'nome': 'José Vieira', 'codigo': '25632', 's...",4530
1,2021-2607395-PeopleSoft Application Engine-Dom...,,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '...",4531
2,,,[],4532
3,2021-2605708-Microfocus Application Life Cycle...,,"[{'nome': 'Arthur Almeida', 'codigo': '26338',...",4533
4,2021-2605711-Microfocus QTP - UFT Automation T...,,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361...",4534


In [465]:
df_prospects_raw = df_prospects_raw[['prospects', 'id_vaga']]

df_prospects_raw.head(5)

,prospects,id_vaga
0,"[{'nome': 'José Vieira', 'codigo': '25632', 's...",4530
1,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '...",4531
2,[],4532
3,"[{'nome': 'Arthur Almeida', 'codigo': '26338',...",4533
4,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361...",4534


## Tratamendo de dados

In [466]:
df_prospects_exploded = df_prospects_raw.explode("prospects").reset_index(drop=True)

df_prospects_expanded = pd.json_normalize(df_prospects_exploded["prospects"])

df_prospects = pd.concat([df_prospects_exploded[["id_vaga"]], df_prospects_expanded], axis=1)

df_prospects['contratado'] = df_prospects['situacao_candidado'].fillna('').apply(lambda x: 1 if 'Contratado' in x else 0)

df_prospects = df_prospects.rename(columns={'codigo': 'id_candidato'})

df_prospects = df_prospects[['id_vaga', 'id_candidato', 'contratado']]

df_prospects = df_prospects[df_prospects['id_candidato'].notnull()]

df_prospects['id_candidato'] = df_prospects['id_candidato'].astype('int64')

df_prospects['id_vaga'] = df_prospects['id_vaga'].astype('int64')

df_prospects.head()

,id_vaga,id_candidato,contratado
0,4530,25632,0
1,4530,25529,0
2,4531,25364,1
3,4531,25360,0
5,4533,26338,1


# Vagas

## Lendo e carregando o arquivo json em um dataframe

In [467]:
with open('C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/Base de dados/vagas.json', 'r', encoding='utf-8') as f:
    dados_vagas = json.load(f)

lista_vagas = []

for vaga_id, vaga_info in dados_vagas.items():
    vaga_info['id_vaga'] = vaga_id
    lista_vagas.append(vaga_info)

df_vagas_raw = pd.DataFrame(lista_vagas)

df_vagas_raw.head()

,informacoes_basicas,perfil_vaga,beneficios,id_vaga
0,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '...",5185
1,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '...",5184
2,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '...",5183
3,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '- p/ mês (168h)', 'valor_comp...",5182
4,"{'data_requicisao': '04-05-2021', 'limite_espe...","{'pais': 'Brasil', 'estado': 'São Paulo', 'cid...","{'valor_venda': '-', 'valor_compra_1': 'R$', '...",5181


In [468]:
df_vagas_raw = pd.json_normalize(df_vagas_raw.to_dict(orient='records'))

df_vagas_raw.head()

,id_vaga,informacoes_basicas.data_requicisao,informacoes_basicas.limite_esperado_para_contratacao,informacoes_basicas.titulo_vaga,informacoes_basicas.vaga_sap,informacoes_basicas.cliente,informacoes_basicas.solicitante_cliente,informacoes_basicas.empresa_divisao,informacoes_basicas.requisitante,informacoes_basicas.analista_responsavel,informacoes_basicas.tipo_contratacao,informacoes_basicas.prazo_contratacao,informacoes_basicas.objetivo_vaga,informacoes_basicas.prioridade_vaga,informacoes_basicas.origem_vaga,informacoes_basicas.superior_imediato,informacoes_basicas.nome,informacoes_basicas.telefone,perfil_vaga.pais,perfil_vaga.estado,perfil_vaga.cidade,perfil_vaga.bairro,perfil_vaga.regiao,perfil_vaga.local_trabalho,perfil_vaga.vaga_especifica_para_pcd,perfil_vaga.faixa_etaria,perfil_vaga.horario_trabalho,perfil_vaga.nivel profissional,perfil_vaga.nivel_academico,perfil_vaga.nivel_ingles,perfil_vaga.nivel_espanhol,perfil_vaga.outro_idioma,perfil_vaga.areas_atuacao,perfil_vaga.principais_atividades,perfil_vaga.competencia_tecnicas_e_comportamentais,perfil_vaga.demais_observacoes,perfil_vaga.viagens_requeridas,perfil_vaga.equipamentos_necessarios,beneficios.valor_venda,beneficios.valor_compra_1,beneficios.valor_compra_2,informacoes_basicas.data_inicial,informacoes_basicas.data_final,perfil_vaga.habilidades_comportamentais_necessarias,informacoes_basicas.nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN


## Renomeando colunas vindas do arquivo

In [469]:
for x in df_vagas_raw.columns:

    if 'informacoes_basicas.' in x:

        x_replaced = x.replace('informacoes_basicas.', '').replace(' ', '_').strip()

        df_vagas_raw = df_vagas_raw.rename(columns={x: x_replaced})

    if 'perfil_vaga.' in x:

        x_replaced2 = x.replace('perfil_vaga.', '').replace(' ', '_').strip()

        df_vagas_raw = df_vagas_raw.rename(columns={x: x_replaced2})

    if 'beneficios.' in x:

        x_replaced3 = x.replace('beneficios.', '').replace(' ', '_').strip()

        df_vagas_raw = df_vagas_raw.rename(columns={x: x_replaced3})

df_vagas_raw.head()

,id_vaga,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,valor_venda,valor_compra_1,valor_compra_2,data_inicial,data_final,habilidades_comportamentais_necessarias,nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN


In [470]:
df_vagas_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14081 entries, 0 to 14080
Data columns (total 45 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   id_vaga                                  14081 non-null  object
 1   data_requicisao                          14081 non-null  object
 2   limite_esperado_para_contratacao         14081 non-null  object
 3   titulo_vaga                              14081 non-null  object
 4   vaga_sap                                 14081 non-null  object
 5   cliente                                  14081 non-null  object
 6   solicitante_cliente                      14081 non-null  object
 7   empresa_divisao                          14081 non-null  object
 8   requisitante                             14081 non-null  object
 9   analista_responsavel                     14081 non-null  object
 10  tipo_contratacao                         14081 non-null  o

## Selecionando colunas

In [471]:
colunas_vagas = [
    'id_vaga'
    ,'vaga_especifica_para_pcd'
    ,'faixa_etaria'
    ,'nivel_profissional'
    ,'nivel_academico'
    ,'nivel_ingles'
    ,'nivel_espanhol'
    ,'outro_idioma'
    ,'areas_atuacao'
    ,'viagens_requeridas'
]

df_vagas = df_vagas_raw[colunas_vagas]

## Formatando coluna outro_idioma

In [472]:
df_vagas['outro_idioma'] = df_vagas['outro_idioma'].apply(lambda x: x.replace(' ou ', ' - ou ') + ' -' if ' ou ' in x else x)

df_vagas['outro_idioma'] = df_vagas['outro_idioma'].apply(lambda x: x.replace(x, 'Nenhum') if 'Português' in x else x)

for x in ['Avançado', 'Fluente', 'Básico']:

    df_vagas['outro_idioma'] = df_vagas['outro_idioma'].replace(x, 'Nenhum')

C:\Users\Ramon\AppData\Local\Temp\ipykernel_9924\4232040934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vagas['outro_idioma'] = df_vagas['outro_idioma'].apply(lambda x: x.replace(' ou ', ' - ou ') + ' -' if ' ou ' in x else x)
C:\Users\Ramon\AppData\Local\Temp\ipykernel_9924\4232040934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vagas['outro_idioma'] = df_vagas['outro_idioma'].apply(lambda x: x.replace(x, 'Nenhum') if 'Português' in x else x)
C:\Users\Ramon\AppData\Local\Temp\ipykernel

In [473]:
df_vagas['outro_idioma'] = df_vagas['outro_idioma'].str.split('ou ')

df_vagas = df_vagas.explode('outro_idioma').reset_index(drop=True)

C:\Users\Ramon\AppData\Local\Temp\ipykernel_9924\2729617689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vagas['outro_idioma'] = df_vagas['outro_idioma'].str.split('ou ')


In [474]:
df_vagas['outro_idioma'] = df_vagas['outro_idioma'].str.strip().apply(lambda x: x + ' Básico' if isinstance(x, str) and x.endswith(' -') else x).str.replace(' - ', ' ')

In [475]:
df_vagas['outro_idioma'] = df_vagas['outro_idioma'].replace('', 'Nenhum')

df_vagas[['lingua_outro_idioma', 'nivel_outro_idioma']] = df_vagas['outro_idioma'].apply(lambda x: pd.Series([x, x]) if x == 'Nenhum' else pd.Series(x.split(' ', 1)))

df_vagas['nivel_outro_idioma'] = df_vagas['nivel_outro_idioma'].str.replace('Desejável', 'Básico')

df_vagas['outro_idioma'] = df_vagas['lingua_outro_idioma'].apply(lambda x: 0 if x == 'Nenhum' else 1)

In [476]:
df_vagas[df_vagas['outro_idioma']==1].head()

,id_vaga,vaga_especifica_para_pcd,faixa_etaria,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,viagens_requeridas,lingua_outro_idioma,nivel_outro_idioma
123,5062,Não,De: Até:,Analista,Ensino Superior Completo,Fluente,Fluente,1,TI - Projetos-,Não,Mandarim,Básico
124,5062,Não,De: Até:,Analista,Ensino Superior Completo,Fluente,Fluente,1,TI - Projetos-,Não,Russo,Básico
2888,10296,Não,De: Até:,Pleno,Ensino Superior Completo,Fluente,,1,Gestão e Alocação de Recursos de TI-TI - Siste...,,Francês,Básico
3544,12224,,De: Até:,Sênior,Ensino Superior Completo,Básico,Básico,1,TI - Projetos-,Não,Francês,Intermediário
4376,11380,Não,De: Até:,Pleno,Ensino Superior Completo,Avançado,Intermediário,1,Novos Negócios e Parcerias-,,Mandarim,Básico


## Separando linha a linha as áreas de atuação

In [477]:
df_vagas['areas_atuacao'] = df_vagas['areas_atuacao'].str.replace('-TI - ', ',TI - ').str[:-1]

In [478]:
df_vagas.shape

(14082, 12)

In [479]:
df_vagas['areas_atuacao'] = df_vagas['areas_atuacao'].str.split(',')

df_vagas = df_vagas.explode('areas_atuacao').reset_index(drop=True)

df_vagas['areas_atuacao'] = df_vagas['areas_atuacao'].str.strip()

In [480]:
df_vagas.shape

(14313, 12)

## Aplicando transformações nas colunas selecionadas

In [481]:
df_vagas['vaga_especifica_para_pcd'] = df_vagas['vaga_especifica_para_pcd'].apply(lambda x: 0 if x == 'Não' else 1)

df_vagas['faixa_etaria'] = df_vagas['faixa_etaria'].str.replace('De: Até:', 'De: 1 Até: 100')

df_vagas['idade_minima'] = df_vagas['faixa_etaria'].str[:6]

df_vagas['idade_minima'] = df_vagas['idade_minima'].str.replace('De:', '').str.strip().astype('int')

df_vagas['idade_maxima'] = df_vagas['faixa_etaria'].str[-6:]

df_vagas['idade_maxima'] = df_vagas['idade_maxima'].str[3:].str.strip().astype('int')

df_vagas['nivel_profissional'] = df_vagas['nivel_profissional'].map(dict_nivel_profissional)

df_vagas['nivel_academico'] = df_vagas['nivel_academico'].map(dict_nivel_escolaridade)

df_vagas['nivel_ingles'] = df_vagas['nivel_ingles'].map(dict_nivel_linguagem)

df_vagas['nivel_espanhol'] = df_vagas['nivel_espanhol'].replace('', 'Nenhum')

df_vagas['nivel_espanhol'] = df_vagas['nivel_espanhol'].map(dict_nivel_linguagem)

df_vagas['id_vaga'] = df_vagas['id_vaga'].astype('int64')

df_vagas['nivel_outro_idioma'] = df_vagas['nivel_outro_idioma'].map(dict_nivel_linguagem)

## Excluindo colunas

In [482]:
df_vagas = df_vagas.drop(columns={'faixa_etaria', 'viagens_requeridas'})

## Renomeando e reorndenando colunas

In [483]:
df_vagas = df_vagas.rename(columns={
    'vaga_especifica_para_pcd': 'vaga_para_pcd'
    ,'nivel_profissional': 'nivel_profissional_vaga'
    ,'nivel_academico': 'nivel_academico_vaga'
    ,'nivel_ingles': 'nivel_ingles_vaga'
    ,'nivel_espanhol': 'nivel_espanhol_vaga'
    ,'areas_atuacao': 'area_atuacao_vaga'
    ,'idade_minima': 'idade_minima_vaga'
    ,'idade_maxima': 'idade_maxima_vaga'
    ,'outro_idioma': 'flg_outro_idioma_vaga'
    ,'lingua_outro_idioma': 'outro_idioma_vaga'
    ,'nivel_outro_idioma': 'nivel_outro_idioma_vaga'
})

In [484]:
df_vagas = df_vagas[[
    'id_vaga'
    ,'vaga_para_pcd'
    ,'idade_minima_vaga'
    ,'idade_maxima_vaga'
    ,'area_atuacao_vaga'
    ,'nivel_academico_vaga'
    ,'nivel_profissional_vaga'
    ,'nivel_ingles_vaga'
    ,'nivel_espanhol_vaga'
    ,'flg_outro_idioma_vaga'
    ,'outro_idioma_vaga'
    ,'nivel_outro_idioma_vaga'
]]

In [485]:
df_vagas[df_vagas['flg_outro_idioma_vaga']==1].head(15)

,id_vaga,vaga_para_pcd,idade_minima_vaga,idade_maxima_vaga,area_atuacao_vaga,nivel_academico_vaga,nivel_profissional_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,flg_outro_idioma_vaga,outro_idioma_vaga,nivel_outro_idioma_vaga
123,5062,0,1,100,TI - Projetos,10,7,5,5,1,Mandarim,1
124,5062,0,1,100,TI - Projetos,10,7,5,5,1,Russo,1
3025,10296,0,1,100,Gestão e Alocação de Recursos de TI,10,8,5,0,1,Francês,1
3026,10296,0,1,100,TI - Sistemas e Ferramentas,10,8,5,0,1,Francês,1
3729,12224,1,1,100,TI - Projetos,10,9,1,1,1,Francês,2
4577,11380,0,1,100,Novos Negócios e Parcerias,10,8,4,2,1,Mandarim,1
4578,11379,0,1,100,Novos Negócios e Parcerias,10,8,4,2,1,Mandarim,1
4579,11378,0,1,100,Novos Negócios e Parcerias,10,8,4,2,1,Mandarim,1
4580,11377,0,1,100,Novos Negócios e Parcerias,10,8,4,2,1,Mandarim,1
4581,11376,0,1,100,Novos Negócios e Parcerias,10,8,4,2,1,Mandarim,1


# Candidatos

## Lendo e carregando o arquivo json em um dataframe pandas

In [486]:
with open('C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/Base de dados/applicants.json', 'r', encoding='utf-8') as f:
    dados_candidatos = json.load(f)

lista_candidatos = []

for candidato_id, candidato_info in dados_candidatos.items():
    candidato_info['id_candidato'] = candidato_id
    lista_candidatos.append(candidato_info)

df_candidatos_raw = pd.DataFrame(lista_candidatos)

df_candidatos_raw = df_candidatos_raw.drop(columns={'cv_pt', 'cv_en', 'infos_basicas'})

In [487]:
df_candidatos_raw = pd.json_normalize(df_candidatos_raw.to_dict(orient='records'))

df_candidatos_raw.head()

,id_candidato,informacoes_pessoais.data_aceite,informacoes_pessoais.nome,informacoes_pessoais.cpf,informacoes_pessoais.fonte_indicacao,informacoes_pessoais.email,informacoes_pessoais.email_secundario,informacoes_pessoais.data_nascimento,informacoes_pessoais.telefone_celular,informacoes_pessoais.telefone_recado,informacoes_pessoais.sexo,informacoes_pessoais.estado_civil,informacoes_pessoais.pcd,informacoes_pessoais.endereco,informacoes_pessoais.skype,informacoes_pessoais.url_linkedin,informacoes_pessoais.facebook,informacoes_profissionais.titulo_profissional,informacoes_profissionais.area_atuacao,informacoes_profissionais.conhecimentos_tecnicos,informacoes_profissionais.certificacoes,informacoes_profissionais.outras_certificacoes,informacoes_profissionais.remuneracao,informacoes_profissionais.nivel_profissional,formacao_e_idiomas.nivel_academico,formacao_e_idiomas.nivel_ingles,formacao_e_idiomas.nivel_espanhol,formacao_e_idiomas.outro_idioma,formacao_e_idiomas.instituicao_ensino_superior,formacao_e_idiomas.cursos,formacao_e_idiomas.ano_conclusao,informacoes_pessoais.download_cv,informacoes_profissionais.qualificacoes,informacoes_profissionais.experiencias,formacao_e_idiomas.outro_curso,cargo_atual.id_ibrati,cargo_atual.email_corporativo,cargo_atual.cargo_atual,cargo_atual.projeto_atual,cargo_atual.cliente,cargo_atual.unidade,cargo_atual.data_admissao,cargo_atual.data_ultima_promocao,cargo_atual.nome_superior_imediato,cargo_atual.email_superior_imediato
0,31000,Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,,,,,,,,,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,,Feminino,Solteiro,Não,são paulo,,,,Analista Administrativo,Administrativa,,,,1900,,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,,Feminino,Solteiro,Não,são paulo,,,,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,Cadastro anterior ao registro de aceite,Thiago Barbosa,,Site de Empregos: Infojobs,thiago_barbosa@hotmail.com,,08-05-1992,(11) 98100-1727,,Feminino,Casado,Não,são paulo,,,,Área administrativa,Administrativa,,,,"1100,00",,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,Cadastro anterior ao registro de aceite,Diogo das Neves,,:,diogo_das_neves@hotmail.com,,31-12-1969,(11) 92517-2678,,,,,,,,,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Renomeando colunas vindas do arquivo

In [488]:
for x in df_candidatos_raw.columns:

    if 'informacoes_pessoais.' in x:

        x_replaced = x.replace('informacoes_pessoais.', '').replace(' ', '_').strip()

        df_candidatos_raw = df_candidatos_raw.rename(columns={x: x_replaced})

    if 'informacoes_profissionais.' in x:

        x_replaced2 = x.replace('informacoes_profissionais.', '').replace(' ', '_').strip()

        df_candidatos_raw = df_candidatos_raw.rename(columns={x: x_replaced2})

    if 'formacao_e_idiomas.' in x:

        x_replaced3 = x.replace('formacao_e_idiomas.', '').replace(' ', '_').strip()

        df_candidatos_raw = df_candidatos_raw.rename(columns={x: x_replaced3})

    if 'cargo_atual.' in x:

        x_replaced3 = x.replace('cargo_atual.', '').replace(' ', '_').strip()

        df_candidatos_raw = df_candidatos_raw.rename(columns={x: x_replaced3})

In [489]:
df_candidatos_raw

,id_candidato,data_aceite,nome,cpf,fonte_indicacao,email,email_secundario,data_nascimento,telefone_celular,telefone_recado,sexo,estado_civil,pcd,endereco,skype,url_linkedin,facebook,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,instituicao_ensino_superior,cursos,ano_conclusao,download_cv,qualificacoes,experiencias,outro_curso,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,,,,,,,,,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,,Feminino,Solteiro,Não,são paulo,,,,Analista Administrativo,Administrativa,,,,1900,,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,,Feminino,Solteiro,Não,são paulo,,,,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,Cadastro anterior ao registro de aceite,Thiago Barbosa,,Site de Empregos: Infojobs,thiago_barbosa@hotmail.com,,08-05-1992,(11) 98100-1727,,Feminino,Casado,Não,são paulo,,,,Área administrativa,Administrativa,,,,"1100,00",,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,Cadastro anterior ao registro de aceite,Diogo das Neves,,:,diogo_das_neves@hotmail.com,,31-12-1969,(11) 92517-2678,,,,,,,,,,,,,,,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42477,5995,Cadastro anterior ao registro de aceite,Gabrielly Novaes,,Outros: Importação,gabrielly_novaes@gmail.com,,0000-00-00,(11) 92622-2088,,,,,,,,,Crm / Líder de Projetos,,,,,0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42478,5996,Cadastro anterior ao registro de aceite,Sr. Gabriel Rios,,Outros: Importação,sr._gabriel_rios@gmail.com,,0000-00-00,(41) 93009-5141,,,,,,,,,Crm,,,,,0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42479,5997,Cadastro anterior ao registro de aceite,Dr. Mathias da Cunha,,Outros: Importação,dr._mathias_da_cunha@gmail.com,,0000-00-00,(11) 97837-3941,,,,,,,,,Crm,,,,,0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42480,5998,Cadastro anterior ao registro de aceite,Bruna Vieira,,Outros: Importação,bruna_vieira@hotmail.com,,0000-00-00,(61) 96806-8075,,,,,,,,,Crm,,,,,0,,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [490]:
df_candidatos_raw['cargo_atual'].drop_duplicates()

0                                 NaN
544                                  
1501          Analista de Projetos II
1647                Consultor (a) SAP
3089     Assistente Administrativo SR
                     ...             
32277             Analista Financeiro
32287          Programador (a) Sênior
32292         Auxiliar de Faturamento
39272       Desenvolvedor Programador
40018           Analista de Marketing
Name: cargo_atual, Length: 62, dtype: object

## Selecionando colunas e filtrando o dataframe

In [491]:
colunas_candidatos = [
    'id_candidato'
    ,'data_nascimento'
    ,'pcd'
    ,'area_atuacao'
    ,'nivel_profissional'
    ,'nivel_academico'
    ,'nivel_ingles'
    ,'nivel_espanhol'
    ,'outro_idioma'
    ,'certificacoes'
    ,'outras_certificacoes'
    ,'cargo_atual'
]

df_candidatos = df_candidatos_raw[colunas_candidatos]

df_candidatos = df_candidatos.replace('', np.nan)

## Formatando coluna outro_idioma

In [492]:
df_candidatos['outro_idioma'] = df_candidatos['outro_idioma'].apply(lambda x: 'Nenhum - Nenhum' if x == '-' or 'Português' in x else x)

df_candidatos['outro_idioma'] = df_candidatos['outro_idioma'].apply(lambda x: x + ' Básico' if x[-2:] == ' -' else x)

df_candidatos[['lingua_outro_idioma', 'nivel_outro_idioma']] = df_candidatos['outro_idioma'].str.split(' - ', expand=True)

df_candidatos['outro_idioma'] = df_candidatos['lingua_outro_idioma'].apply(lambda x: 0 if x == 'Nenhum' else 1)

## Aplicando transformações nas colunas escolhidas

In [493]:
df_candidatos['data_nascimento'] = df_candidatos['data_nascimento'].str[6:].replace('0-00', 0).fillna(0).astype('int')

df_candidatos['idade'] = date.today().year - df_candidatos['data_nascimento']

df_candidatos['nivel_profissional'] = df_candidatos['nivel_profissional'].map(dict_nivel_profissional)

df_candidatos['pcd'] = df_candidatos['pcd'].apply(lambda x: 0 if x == 'Não' else 1)

df_candidatos['nivel_ingles'] = df_candidatos['nivel_ingles'].map(dict_nivel_linguagem)

df_candidatos['nivel_espanhol'] = df_candidatos['nivel_espanhol'].map(dict_nivel_linguagem)

df_candidatos['nivel_academico'] = df_candidatos['nivel_academico'].str.strip().map(dict_nivel_escolaridade)

df_candidatos['id_candidato'] = df_candidatos['id_candidato'].astype('int64')

df_candidatos['nivel_outro_idioma'] = df_candidatos['nivel_outro_idioma'].map(dict_nivel_linguagem)

df_candidatos['flg_certificacao_candidato'] = np.where(df_candidatos['certificacoes'].notnull(), 1, 0)

df_candidatos['certificacoes'] = df_candidatos['certificacoes'].fillna('Nenhuma')

df_candidatos['flg_outras_certificacoes_candidato'] = np.where(df_candidatos['outras_certificacoes'].notnull(), 1, 0)

df_candidatos['flg_candidato_empregado'] = np.where(df_candidatos['cargo_atual'].notnull(), 1, 0)

df_candidatos['cargo_atual'] = df_candidatos['cargo_atual'].fillna('Desempregado')

In [494]:
df_candidatos.head(10)

,id_candidato,data_nascimento,pcd,area_atuacao,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,certificacoes,outras_certificacoes,cargo_atual,lingua_outro_idioma,nivel_outro_idioma,idade,flg_certificacao_candidato,flg_outras_certificacoes_candidato,flg_candidato_empregado
0,31000,0,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,2025,0,0,0
1,31001,1994,0,Administrativa,NaN,8.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,31,0,0,0
2,31002,1988,0,Administrativa,NaN,10.0,2.0,1.0,0,"MS [77-418] MOS: Microsoft Office Word 2013, M...",NaN,Desempregado,Nenhum,0,37,1,0,0
3,31003,1992,0,Administrativa,NaN,8.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,33,0,0,0
4,31004,1969,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,56,0,0,0
5,31005,1979,1,Comercial,NaN,7.0,1.0,1.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,46,0,0,0
6,31006,1990,1,Financeira/Controladoria,NaN,9.0,0.0,2.0,0,Nenhuma,"SAP, DN4,MICROSIGA,TOTVS",Desempregado,Nenhum,0,35,0,1,0
7,31007,1995,0,"Administrativa, Comercial, Financeira/Controla...",NaN,10.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,30,0,0,0
8,31008,0,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,2025,0,0,0
9,31009,0,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,2025,0,0,0


## Separando linha a linha as áreas de atuação

In [495]:
df_candidatos['area_atuacao'] = df_candidatos['area_atuacao'].str.split(',')

df_candidatos = df_candidatos.explode('area_atuacao').reset_index(drop=True)

df_candidatos['area_atuacao'] = df_candidatos['area_atuacao'].str.strip()

In [496]:
df_candidatos.head(10)

,id_candidato,data_nascimento,pcd,area_atuacao,nivel_profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,certificacoes,outras_certificacoes,cargo_atual,lingua_outro_idioma,nivel_outro_idioma,idade,flg_certificacao_candidato,flg_outras_certificacoes_candidato,flg_candidato_empregado
0,31000,0,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,2025,0,0,0
1,31001,1994,0,Administrativa,NaN,8.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,31,0,0,0
2,31002,1988,0,Administrativa,NaN,10.0,2.0,1.0,0,"MS [77-418] MOS: Microsoft Office Word 2013, M...",NaN,Desempregado,Nenhum,0,37,1,0,0
3,31003,1992,0,Administrativa,NaN,8.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,33,0,0,0
4,31004,1969,1,NaN,NaN,NaN,NaN,NaN,0,Nenhuma,NaN,Desempregado,Nenhum,0,56,0,0,0
5,31005,1979,1,Comercial,NaN,7.0,1.0,1.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,46,0,0,0
6,31006,1990,1,Financeira/Controladoria,NaN,9.0,0.0,2.0,0,Nenhuma,"SAP, DN4,MICROSIGA,TOTVS",Desempregado,Nenhum,0,35,0,1,0
7,31007,1995,0,Administrativa,NaN,10.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,30,0,0,0
8,31007,1995,0,Comercial,NaN,10.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,30,0,0,0
9,31007,1995,0,Financeira/Controladoria,NaN,10.0,0.0,0.0,0,Nenhuma,NaN,Desempregado,Nenhum,0,30,0,0,0


## Excluindo colunas

In [497]:
df_candidatos = df_candidatos.drop(columns={'data_nascimento', 'outras_certificacoes'}, axis=1)

## Renomeando e reordenando colunas

In [498]:
df_candidatos = df_candidatos.rename(columns={
    'pcd': 'candidato_pcd'
    ,'area_atuacao': 'area_atuacao_candidato'
    ,'nivel_profissional': 'nivel_profissional_candidato'
    ,'nivel_academico': 'nivel_academico_candidato'
    ,'nivel_ingles': 'nivel_ingles_candidato'
    ,'nivel_espanhol': 'nivel_espanhol_candidato'
    ,'idade': 'idade_candidato'
    ,'outro_idioma': 'flg_outro_idioma_candidato'
    ,'lingua_outro_idioma': 'outro_idioma_candidato'
    ,'nivel_outro_idioma': 'nivel_outro_idioma_candidato'
    ,'certificacoes': 'certificacoes_candidato'
    ,'cargo_atual': 'cargo_atual_candidato'
})

In [499]:
df_candidatos = df_candidatos[[
    'id_candidato'
    ,'idade_candidato'
    ,'candidato_pcd'
    ,'area_atuacao_candidato'
    ,'nivel_academico_candidato'
    ,'nivel_profissional_candidato'
    ,'nivel_ingles_candidato'
    ,'nivel_espanhol_candidato'
    ,'flg_outro_idioma_candidato'
    ,'outro_idioma_candidato'
    ,'nivel_outro_idioma_candidato'
    ,'flg_certificacao_candidato'
    ,'certificacoes_candidato'
    ,'flg_outras_certificacoes_candidato'
    ,'flg_candidato_empregado'
    ,'cargo_atual_candidato'
]]

# Dataframe final

## Juntando dataframes para cruzar os dados

In [500]:
df = pd.merge(df_candidatos, df_prospects, on='id_candidato', how='left')
df = pd.merge(df, df_vagas, on='id_vaga', how='left')
df = df[df['idade_maxima_vaga'].notnull()]

In [501]:
df.shape

(56623, 29)

## Checando valores nulos

In [502]:
nulos = []

for x in df.columns:

    linhas = len(df[df[x].isnull()])

    if linhas > 0:

        print(f'Coluna {x}: {linhas} linhas.\n')

    if linhas > 0:
        
        nulos.append(linhas)

if not nulos:

    print('Não há valores nulos na base.')

Coluna area_atuacao_candidato: 35367 linhas.

Coluna nivel_academico_candidato: 34562 linhas.

Coluna nivel_profissional_candidato: 56240 linhas.

Coluna nivel_ingles_candidato: 35246 linhas.

Coluna nivel_espanhol_candidato: 35637 linhas.



## Normalizando dados

In [503]:
# Coluna idade_candidato

df['idade_candidato'] = df['idade_candidato'].replace(2025, np.nan)

df['idade_candidato'] = df['idade_candidato'].fillna((df['idade_minima_vaga'] + df['idade_maxima_vaga']).div(2))

df['idade_candidato'] = df['idade_candidato'].astype('int64')

# Coluna area_atuacao_candidato

df['area_atuacao_candidato'] = df['area_atuacao_candidato'].fillna('Nao informado')

# Coluna candidato_pcd

df['candidato_pcd'] = df['candidato_pcd'].fillna(df['vaga_para_pcd'])

df['candidato_pcd'] = df['candidato_pcd'].astype('int64')

for col in ['nivel_academico_candidato', 'nivel_profissional_candidato', 'nivel_ingles_candidato', 'nivel_espanhol_candidato']:

    df[f'{col}_is_missing'] = df[col].isnull().astype('int64')

# Colunas nivel_academico_candidato, nivel_profissional_candidato, nivel_ingles_candidato, nivel_espanhol_candidato

for x in ['nivel_academico_candidato', 'nivel_profissional_candidato', 'nivel_ingles_candidato', 'nivel_espanhol_candidato']:

    col_vaga = x.replace('candidato', 'vaga')

    min_value_contratado = df[col_vaga].mul(0.5).astype('int64')

    max_value_contratado = df[col_vaga].mul(1.5).astype('int64')

    max_value_contratado = np.where(max_value_contratado <= min_value_contratado, max_value_contratado + 1, max_value_contratado)

    max_value_nao_contratado = df[col_vaga].mul(1.5).astype('int64')#.clip(lower=1).astype('int64')

    max_value_nao_contratado = np.where(max_value_nao_contratado == 0, 1, max_value_nao_contratado)

    max_value_nao_contratado = np.where(max_value_nao_contratado == 1, 2, max_value_nao_contratado)

    df[x] = np.where(
        df[x].isnull()
        ,np.where(
            df['contratado'] == 1
            ,np.random.randint(min_value_contratado, max_value_contratado)
            ,np.random.randint(1, max_value_nao_contratado)
        )
        ,df[x]
    )

# Coluna flg_outro_idioma_candidato_nao_informado

df['flg_outro_idioma_candidato_nao_informado'] = np.where(df['flg_outro_idioma_candidato'].isnull(), 1, 0)
    
# Coluna flg_outro_idioma_candidato

df['flg_outro_idioma_candidato'] = df['flg_outro_idioma_candidato'].fillna(0)
    
# Coluna outro_idioma_candidato

df['outro_idioma_candidato'] = df['outro_idioma_candidato'].fillna('Nao informado')
    
# Coluna nivel_outro_idioma_candidato

df['nivel_outro_idioma_candidato'] = df['nivel_outro_idioma_candidato'].fillna(0)

## Validando normalização

In [504]:
nulos = []

for x in df.columns:

    linhas = len(df[df[x].isnull()])

    if linhas > 0:

        print(f'Coluna {x}: {linhas} linhas.\n')

    if linhas > 0:
        
        nulos.append(linhas)

if not nulos:

    print('Não há valores nulos na base.')

Não há valores nulos na base.


In [505]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56623 entries, 0 to 76698
Data columns (total 34 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   id_candidato                              56623 non-null  int64  
 1   idade_candidato                           56623 non-null  int64  
 2   candidato_pcd                             56623 non-null  int64  
 3   area_atuacao_candidato                    56623 non-null  object 
 4   nivel_academico_candidato                 56623 non-null  float64
 5   nivel_profissional_candidato              56623 non-null  float64
 6   nivel_ingles_candidato                    56623 non-null  float64
 7   nivel_espanhol_candidato                  56623 non-null  float64
 8   flg_outro_idioma_candidato                56623 non-null  int64  
 9   outro_idioma_candidato                    56623 non-null  object 
 10  nivel_outro_idioma_candidato           

In [506]:
df = df.reset_index(drop=True)
df_vagas = df_vagas.reset_index(drop=True)

In [507]:
df_keys = df[['id_vaga', 'id_candidato']]

In [508]:
df = df.drop(columns={'id_vaga', 'id_candidato'}, axis=1)

In [509]:
caminho_csv = 'C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/scripts'

df.to_csv(f'{caminho_csv}/admission_base_de_dados.csv', index=False)

df_vagas.to_csv(f'{caminho_csv}/vagas.csv', index=False)

print(f'DataFrames salvos com sucesso em: {caminho_csv}')

DataFrames salvos com sucesso em: C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/scripts


In [510]:
df.head()

,idade_candidato,candidato_pcd,area_atuacao_candidato,nivel_academico_candidato,nivel_profissional_candidato,nivel_ingles_candidato,nivel_espanhol_candidato,flg_outro_idioma_candidato,outro_idioma_candidato,nivel_outro_idioma_candidato,flg_certificacao_candidato,certificacoes_candidato,flg_outras_certificacoes_candidato,flg_candidato_empregado,cargo_atual_candidato,contratado,vaga_para_pcd,idade_minima_vaga,idade_maxima_vaga,area_atuacao_vaga,nivel_academico_vaga,nivel_profissional_vaga,nivel_ingles_vaga,nivel_espanhol_vaga,flg_outro_idioma_vaga,outro_idioma_vaga,nivel_outro_idioma_vaga,nivel_academico_candidato_is_missing,nivel_profissional_candidato_is_missing,nivel_ingles_candidato_is_missing,nivel_espanhol_candidato_is_missing,flg_outro_idioma_candidato_nao_informado
0,50,1,Nao informado,14.0,1.0,1.0,1.0,0,Nenhum,0,0,Nenhuma,0,0,Desempregado,0.0,0.0,1.0,100.0,TI - Projetos,10.0,7.0,1.0,1.0,0.0,Nenhum,0.0,1,1,1,1,0
1,50,1,Nao informado,12.0,1.0,1.0,1.0,0,Nenhum,0,0,Nenhuma,0,0,Desempregado,0.0,0.0,1.0,100.0,TI - Projetos,9.0,7.0,1.0,1.0,0.0,Nenhum,0.0,1,1,1,1,0
2,31,0,Administrativa,8.0,4.0,0.0,0.0,0,Nenhum,0,0,Nenhuma,0,0,Desempregado,0.0,0.0,1.0,100.0,TI - Projetos,9.0,7.0,1.0,1.0,0.0,Nenhum,0.0,0,1,0,0,0
3,37,0,Administrativa,10.0,9.0,2.0,1.0,0,Nenhum,0,1,"MS [77-418] MOS: Microsoft Office Word 2013, M...",0,0,Desempregado,0.0,0.0,1.0,100.0,TI - Projetos,9.0,7.0,1.0,1.0,0.0,Nenhum,0.0,0,1,0,0,0
4,33,0,Administrativa,8.0,4.0,0.0,0.0,0,Nenhum,0,0,Nenhuma,0,0,Desempregado,0.0,0.0,1.0,100.0,TI - Projetos,10.0,7.0,1.0,1.0,0.0,Nenhum,0.0,0,1,0,0,0


# Criando o modelo

## Bibliotecas

In [511]:
# Separar a base entre treino e teste
from sklearn.model_selection import train_test_split

# Criação de classes
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.pipeline import Pipeline

# Normalização de dados
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.impute import KNNImputer

# Modelos
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

# Métricas de validação
from sklearn.metrics import roc_auc_score, classification_report

## Seed

In [512]:
_seed = 1034

## Separando base de treino e teste

In [513]:
target = 'contratado'
features = df.columns.drop(target)

X = df[features]
y = df[target]

In [514]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=_seed, stratify=y)

## Etapas do pipeline

### MinMaxScaler

In [515]:
class MMScaler(BaseEstimator, TransformerMixin):

    def __init__(self, ToBe_Scaled):
        self.ToBe_Scaled = ToBe_Scaled
        self.scaler = MinMaxScaler()

    def fit(self, df, y=None):
        self.scaler.fit(df[self.ToBe_Scaled])
        return self
    
    def transform(self, df):
        
        df_copy = df.copy()
        
        df_copy[self.ToBe_Scaled] = self.scaler.transform(df_copy[self.ToBe_Scaled])

        return df_copy

### OneHotEncoding

In [516]:
class OHEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, ToBe_Encoded):
        self.ToBe_Encoded = ToBe_Encoded
        self.encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        self.feature_names_out_ = None

    def fit(self, df, y=None):
        self.encoder.fit(df[self.ToBe_Encoded])
        self.feature_names_out_ = self.encoder.get_feature_names_out(self.ToBe_Encoded)
        
        return self

    def transform(self, df):
        
        df_copy = df.copy()
        
        encoded_data = self.encoder.transform(df_copy[self.ToBe_Encoded])
        
        df_encoded = pd.DataFrame(
            encoded_data
            ,columns=self.feature_names_out_
            ,index=df_copy.index
        )
        
        df_copy = df_copy.drop(self.ToBe_Encoded, axis=1)
        
        df_final = pd.concat([df_copy, df_encoded], axis=1)
        
        return df_final

### FeatureNameCleaner

In [517]:
class FeatureNameCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, df, y=None):
        return self

    def transform(self, df):
        df_copy = df.copy()
        for x in df_copy.columns:
            df_copy = df_copy.rename(columns={x: unidecode(x).replace(' ', '_').replace('.', '_').replace('/', '_').replace('-', '_').replace('[', '').replace(']', '').replace(':', '').replace('__', '_')})

        return df_copy

### KNNImputer

In [518]:
class DataImputer(BaseEstimator, TransformerMixin):
    
    def __init__(self, ToBe_Imputed, n_neighbors=5):
        self.ToBe_Imputed = ToBe_Imputed
        self.n_neighbors = n_neighbors
        self.imputer = KNNImputer(n_neighbors=self.n_neighbors)

    def fit(self, X, y=None):
        self.imputer.fit(X[self.ToBe_Imputed])
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        imputed_subset = self.imputer.transform(X_copy[self.ToBe_Imputed])
        
        df_imputed_subset = pd.DataFrame(
            imputed_subset 
            ,columns=self.ToBe_Imputed 
            ,index=X_copy.index
        )
        
        X_copy[self.ToBe_Imputed] = df_imputed_subset
        
        return X_copy

### MatchesCreator

In [519]:
class MatchesCreator(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, df, y=None):
        return self

    def transform(self, df):

        df_copy = df.copy()

        df_copy = df_copy.assign(
            match_area_atuacao = np.where(df_copy['area_atuacao_candidato'] == df_copy['area_atuacao_vaga'], 1, 0),
            match_pcd = np.where(df_copy['candidato_pcd'] == df_copy['vaga_para_pcd'], 1, 0),
            match_outro_idioma = np.where(df_copy['outro_idioma_candidato'] == df_copy['outro_idioma_vaga'], 1, 0)
        )

        for col in ['match_area_atuacao', 'match_pcd', 'match_outro_idioma']:
            df_copy[col] = df_copy[col].astype('int64')

        return df_copy

### FeatureCreator

In [520]:
class FeatureCreator(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, df, y=None):
        return self

    def transform(self, df):

        df_copy = df.copy()

        for x in ['nivel_academico_candidato', 'nivel_profissional_candidato', 'nivel_ingles_candidato', 'nivel_espanhol_candidato']:
            if x in df_copy.columns:
                df_copy[x] = df_copy[x].round(0)

        colunas_texto = df_copy.select_dtypes(include='object').columns.tolist()

        for col in [col for col in df_copy.columns if col not in colunas_texto]:
            df_copy[col] = df_copy[col].astype('int64')
        
        return df_copy.assign(
            diff_nivel_profissional = df_copy['nivel_profissional_candidato'] - df_copy['nivel_profissional_vaga'],
            diff_nivel_academico = df_copy['nivel_academico_candidato'] - df_copy['nivel_academico_vaga'],
            diff_nivel_ingles = df_copy['nivel_ingles_candidato'] - df_copy['nivel_ingles_vaga'],
            diff_nivel_espanhol = df_copy['nivel_espanhol_candidato'] - df_copy['nivel_espanhol_vaga'],
            diff_nivel_outro_idioma = df_copy['nivel_outro_idioma_candidato'] - df_copy['nivel_outro_idioma_vaga']
        )

## Colunas a serem normalizadas

In [521]:
colunas_scaler = X_train.select_dtypes(include='int64').columns.tolist()

colunas_encoding = X_train.select_dtypes(include='object').columns.tolist()

colunas_imputer = [
    'nivel_academico_candidato'
    ,'nivel_profissional_candidato'
    ,'nivel_ingles_candidato'
    ,'nivel_espanhol_candidato'
    ,'flg_outro_idioma_candidato'
]

## Pipeline

In [522]:
pipeline = Pipeline([
    ('matches_creator', MatchesCreator())
    ,('one_hot_encoder', OHEncoder(ToBe_Encoded=colunas_encoding))
    ,('name_cleaner', FeatureNameCleaner())
    # ,('knn_imputer', DataImputer(ToBe_Imputed=colunas_imputer))
    ,('feature_creator', FeatureCreator())
    ,('min_max_scaler', MMScaler(ToBe_Scaled=colunas_scaler))
    ,('smote', SMOTE(random_state=_seed))
    ,('classifier', RandomForestClassifier(max_depth=None, min_samples_split=5, random_state=_seed, criterion='entropy'))
])

In [523]:
pipeline2 = Pipeline([
    ('matches_creator', MatchesCreator())
    ,('one_hot_encoder', OHEncoder(ToBe_Encoded=colunas_encoding))
    ,('name_cleaner', FeatureNameCleaner())
    # ,('knn_imputer', DataImputer(ToBe_Imputed=colunas_imputer))
    ,('feature_creator', FeatureCreator())
    ,('min_max_scaler', MMScaler(ToBe_Scaled=colunas_scaler))
    ,('smote', SMOTE(random_state=_seed))
    ,('classifier', KNeighborsClassifier(n_neighbors=3))
])

In [524]:
pipeline3 = Pipeline([
    ('matches_creator', MatchesCreator())
    ,('one_hot_encoder', OHEncoder(ToBe_Encoded=colunas_encoding))
    ,('name_cleaner', FeatureNameCleaner())
    # ,('knn_imputer', DataImputer(ToBe_Imputed=colunas_imputer))
    ,('feature_creator', FeatureCreator())
    ,('min_max_scaler', MMScaler(ToBe_Scaled=colunas_scaler))
    ,('smote', SMOTE(random_state=_seed))
    ,('classifier', XGBClassifier(random_state=_seed, use_label_encoder=False, eval_metric='logloss'))
])

In [525]:
pipeline4 = Pipeline([
    ('matches_creator', MatchesCreator())
    ,('one_hot_encoder', OHEncoder(ToBe_Encoded=colunas_encoding))
    ,('name_cleaner', FeatureNameCleaner())
    ,('knn_imputer', DataImputer(ToBe_Imputed=colunas_imputer))
    ,('feature_creator', FeatureCreator())
    ,('min_max_scaler', MMScaler(ToBe_Scaled=colunas_scaler))
    ,('smote', SMOTE(random_state=_seed))
    ,('classifier', SVC(random_state=_seed, probability=True))
])

## Treinamento do modelo

In [526]:
print('Iniciando o treinamento do pipeline...')
pipeline.fit(X_train, y_train)
print('Treinamento concluído!')

Iniciando o treinamento do pipeline...
Treinamento concluído!


In [527]:
print('Iniciando o treinamento do pipeline...')
pipeline2.fit(X_train, y_train)
print('Treinamento concluído!')

Iniciando o treinamento do pipeline...
Treinamento concluído!


In [528]:
print('Iniciando o treinamento do pipeline...')
pipeline3.fit(X_train, y_train)
print('Treinamento concluído!')

Iniciando o treinamento do pipeline...


c:\Users\Ramon\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [23:26:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Treinamento concluído!


In [529]:
# print('Iniciando o treinamento do pipeline...')
# pipeline4.fit(X_train, y_train)
# print('Treinamento concluído!')

## Previsão do modelo

In [530]:
print('Iniciando avaliação no Conjunto de Teste...')
predictions = pipeline.predict(X_test)
print('Previsão concluída!')

Iniciando avaliação no Conjunto de Teste...
Previsão concluída!


In [531]:
print('Iniciando avaliação no Conjunto de Teste...')
predictions2 = pipeline2.predict(X_test)
print('Previsão concluída!')

Iniciando avaliação no Conjunto de Teste...
Previsão concluída!


In [532]:
print('Iniciando avaliação no Conjunto de Teste...')
predictions3 = pipeline3.predict(X_test)
print('Previsão concluída!')

Iniciando avaliação no Conjunto de Teste...
Previsão concluída!


In [533]:
# print('Iniciando avaliação no Conjunto de Teste...')
# predictions4 = pipeline4.predict(X_test)
# print('Previsão concluída!')

## Validação do modelo

In [534]:
print('-------------------- Relatório de Classificação --------------------\n')
print('-------------------- Teste RandomForest:-----------------\n', classification_report(y_test, predictions),'\n')
print('-------------------- Teste KNeighbors:-------------------\n', classification_report(y_test, predictions2),'\n')
print('-------------------- Teste XGBCBoost:--------------------\n', classification_report(y_test, predictions3),'\n')
# print('-------------------- Teste SVC:--------------------------\n', classification_report(y_test, predictions4))

-------------------- Relatório de Classificação --------------------

-------------------- Teste RandomForest:-----------------
               precision    recall  f1-score   support

         0.0       0.99      1.00      0.99     13479
         1.0       0.98      0.80      0.88       677

    accuracy                           0.99     14156
   macro avg       0.99      0.90      0.94     14156
weighted avg       0.99      0.99      0.99     14156
 

-------------------- Teste KNeighbors:-------------------
               precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     13479
         1.0       0.51      0.77      0.61       677

    accuracy                           0.95     14156
   macro avg       0.75      0.87      0.79     14156
weighted avg       0.97      0.95      0.96     14156
 

-------------------- Teste XGBCBoost:--------------------
               precision    recall  f1-score   support

         0.0       0.99      1.00      0

## Recall do modelo

In [535]:
proba_predictions = pipeline.predict_proba(X_test)[:, 1]
proba_predictions2 = pipeline2.predict_proba(X_test)[:, 1]
proba_predictions3 = pipeline3.predict_proba(X_test)[:, 1]
# proba_predictions4 = pipeline4.predict_proba(X_test)[:, 1]

In [536]:
limiar_otimizado = 0.35

predicoes_com_limiar = (proba_predictions >= limiar_otimizado).astype(int)
predicoes_com_limiar2 = (proba_predictions2 >= limiar_otimizado).astype(int)
predicoes_com_limiar3 = (proba_predictions3 >= limiar_otimizado).astype(int)
# predicoes_com_limiar4 = (proba_predictions4 >= limiar_otimizado).astype(int)

In [537]:
print(f'-------------------- Resultados com Limiar Otimizado de {limiar_otimizado} --------------------\n')
print('-------------------- Teste RandomForest:-----------------\n', classification_report(y_test, predicoes_com_limiar),'\n')
print('-------------------- Teste KNeighbors:-------------------\n', classification_report(y_test, predicoes_com_limiar2),'\n')
print('-------------------- Teste XGBCBoost:--------------------\n', classification_report(y_test, predicoes_com_limiar3),'\n')
# print('-------------------- Teste SVC:--------------------------\n', classification_report(y_test, predicoes_com_limiar4))

-------------------- Resultados com Limiar Otimizado de 0.35 --------------------

-------------------- Teste RandomForest:-----------------
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99     13479
         1.0       0.88      0.85      0.87       677

    accuracy                           0.99     14156
   macro avg       0.94      0.92      0.93     14156
weighted avg       0.99      0.99      0.99     14156
 

-------------------- Teste KNeighbors:-------------------
               precision    recall  f1-score   support

         0.0       0.99      0.96      0.98     13479
         1.0       0.51      0.77      0.61       677

    accuracy                           0.95     14156
   macro avg       0.75      0.87      0.79     14156
weighted avg       0.97      0.95      0.96     14156
 

-------------------- Teste XGBCBoost:--------------------
               precision    recall  f1-score   support

         0.0       0.99    

## Salvando o modelo

In [538]:
import joblib

caminho = 'C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/scripts/ml_model_admissao.joblib'

joblib.dump(pipeline, caminho)

['C:/Users/Ramon/OneDrive/FIAP/Tech Challenges/Modulo 5/scripts/ml_model_admissao.joblib']